In [1]:
from agentic_blocks.utils.tools_utils import create_tool_registry, execute_and_add_tool_responses
from agentic_blocks.utils.tools_utils import execute_pending_tool_calls
from agentic_blocks.utils.tools_utils import langchain_tool_to_openai_format
from agentic_blocks import call_llm, Messages
from deep_research.tools import internet_search, write_todos

In [2]:


tools = [internet_search, write_todos]

tool_registry = create_tool_registry(tools)

langchain_tool_to_openai_format(write_todos)

{'type': 'function',
 'function': {'name': 'write_todos',
  'description': 'Use this tool to create and manage a structured task list for your current work session. This helps you track progress, organize complex tasks, and demonstrate thoroughness to the user.\nIt also helps the user understand the progress of the task and overall progress of their requests.\n\n## When to Use This Tool\nUse this tool proactively in these scenarios:\n\n1. Complex multi-step tasks - When a task requires 3 or more distinct steps or actions\n2. Non-trivial and complex tasks - Tasks that require careful planning or multiple operations\n3. User explicitly requests todo list - When the user directly asks you to use the todo list\n4. User provides multiple tasks - When users provide a list of things to be done (numbered or comma-separated)\n5. After receiving new instructions - Immediately capture user requirements as todos\n6. When you start working on a task - Mark it as in_progress BEFORE beginning work. I

In [3]:
write_todos.args_schema.model_json_schema()

{'$defs': {'Todo': {'description': 'Todo to track.',
   'properties': {'content': {'description': 'The content/description of the todo item',
     'title': 'Content',
     'type': 'string'},
    'status': {'default': 'pending',
     'description': 'Status of the todo item',
     'enum': ['pending', 'in_progress', 'completed'],
     'title': 'Status',
     'type': 'string'}},
   'required': ['content'],
   'title': 'Todo',
   'type': 'object'}},
 'properties': {'todos': {'items': {'$ref': '#/$defs/Todo'},
   'title': 'Todos',
   'type': 'array'}},
 'required': ['todos'],
 'title': 'write_todos',
 'type': 'object'}

In [4]:
research_instructions = """You are an expert researcher. Your job is to conduct thorough research, and then write a polished report.

First use the write_todos tool to create a todo list.

When you have a list of todos, use the internet_search tool to conduct deep research. It will respond to your questions/topics with a detailed answer.

When you think you enough information to write a final report provide the final report.

Here are instructions for writing the final report:

<report_instructions>

CRITICAL: If you make a todo plan - you should note in the plan what language the report should be in so you dont forget!

Note: Write the report in English.

Please create a detailed answer to the overall research brief that:
1. Is well-organized with proper headings (# for title, ## for sections, ### for subsections)
2. Includes specific facts and insights from the research
3. References relevant sources using [Title](URL) format
4. Provides a balanced, thorough analysis. Be as comprehensive as possible, and include all information that is relevant to the overall research question. People are using you for deep research and will expect detailed, comprehensive answers.
5. Includes a "Sources" section at the end with all referenced links

You can structure your report in a number of different ways. Here are some examples:

To answer a question that asks you to compare two things, you might structure your report like this:
1/ intro
2/ overview of topic A
3/ overview of topic B
4/ comparison between A and B
5/ conclusion

To answer a question that asks you to return a list of things, you might only need a single section which is the entire list.
1/ list of things or table of things
Or, you could choose to make each item in the list a separate section in the report. When asked for lists, you don't need an introduction or conclusion.
1/ item 1
2/ item 2
3/ item 3

To answer a question that asks you to summarize a topic, give a report, or give an overview, you might structure your report like this:
1/ overview of topic
2/ concept 1
3/ concept 2
4/ concept 3
5/ conclusion

If you think you can answer the question with a single section, you can do that too!
1/ answer

REMEMBER: Section is a VERY fluid and loose concept. You can structure your report however you think is best, including in ways that are not listed above!
Make sure that your sections are cohesive, and make sense for the reader.

For each section of the report, do the following:
- Use simple, clear language
- Use ## for section title (Markdown format) for each section of the report
- Do NOT ever refer to yourself as the writer of the report. This should be a professional report without any self-referential language. 
- Do not say what you are doing in the report. Just write the report without any commentary from yourself.
- Each section should be as long as necessary to deeply answer the question with the information you have gathered. It is expected that sections will be fairly long and verbose. You are writing a deep research report, and users will expect a thorough answer.
- Use bullet points to list out information when appropriate, but by default, write in paragraph form.

REMEMBER:
The brief and research may be in English, but you need to translate this information to the right language when writing the final answer.
Make sure the final answer report is in English.

Format the report in clear markdown with proper structure and include source references where appropriate.

<Citation Rules>
- Assign each unique URL a single citation number in your text
- End with ### Sources that lists each source with corresponding numbers
- IMPORTANT: Number sources sequentially without gaps (1,2,3,4...) in the final list regardless of which sources you choose
- Each source should be a separate line item in a list, so that in markdown it is rendered as a list.
- Example format:
  [1] Source Title: URL
  [2] Source Title: URL
- Citations are extremely important. Make sure to include these, and pay a lot of attention to getting these right. Users will often use these citations to look into more information.
</Citation Rules>
</report_instructions>

You have access to these tools:

## `write_todos`

Use this to create and manage a structured task list. Pass an array of todo objects, each with:
- `content`: A string describing the task (required)
- `status`: One of "pending", "in_progress", or "completed" (optional, defaults to "pending")

Example: write_todos(todos=[{"content": "Research top restaurants", "status": "pending"}, {"content": "Write report", "status": "pending"}])

## `internet_search`

Use this to run an internet search for a given query. You can specify the number of results, the topic, and whether raw content should be included.
"""

In [5]:
messages = Messages(
    system_prompt=research_instructions,
    user_prompt="Make a summary of the best restaurants in Stockholm"
)

model = "qwen/qwen3-235b-a22b-2507"
#model = "openai/gpt-oss-120b"

In [6]:
has_answer = False

while not has_answer:
    response = call_llm(model=model, messages=messages, tools=tools)
    messages.add_response_message(response)
    #print(messages.get_messages()[-1])
    
    if messages.has_pending_tool_calls():
        for tool_call in messages.get_pending_tool_calls():
            print(f"## Tool call: {tool_call['tool_name']}: {tool_call['arguments']}")
        #print(messages.get_pending_tool_calls())
        tool_responses = execute_pending_tool_calls(messages, tool_registry)
        messages.add_tool_responses(tool_responses)
    else:
        has_answer = True
    
    

## Tool call: write_todos: {'todos': [{'content': 'Research the best restaurants in Stockholm', 'status': 'pending'}, {'content': 'Identify top-rated and highly recommended dining establishments', 'status': 'pending'}, {'content': 'Categorize restaurants by cuisine, price range, and unique features', 'status': 'pending'}, {'content': 'Summarize findings into a comprehensive report', 'status': 'pending'}]}
-------------- Writing todos --------------
- [pending] Research the best restaurants in Stockholm
- [pending] Identify top-rated and highly recommended dining establishments
- [pending] Categorize restaurants by cuisine, price range, and unique features
- [pending] Summarize findings into a comprehensive report
## Tool call: internet_search: {'query': 'best restaurants in Stockholm', 'max_results': 5}
## Tool call: internet_search: {'query': 'Michelin-starred restaurants Stockholm', 'max_results': 5}
## Tool call: internet_search: {'query': 'best traditional Swedish restaurants Stock

In [7]:
print(response.content)

# Best Restaurants in Stockholm: A Comprehensive Guide

Stockholm offers a diverse and sophisticated culinary landscape that ranges from world-renowned fine dining establishments to traditional Swedish eateries and affordable casual spots. The city's restaurant scene reflects both its deep-rooted culinary heritage and its innovative modern approach to Nordic and international cuisine. This guide provides a comprehensive overview of the best dining options in Stockholm, categorized by experience type, cuisine, and budget level.

## Michelin-Starred and Fine Dining Excellence

Stockholm is home to a growing number of Michelin-starred restaurants, with 12 establishments holding stars in the prestigious guide [1]. The city's fine dining scene has gained international recognition, particularly with Frantzén becoming the first restaurant in Stockholm to earn three Michelin stars in 2018, cementing its status as a culinary destination.

**Frantzén** stands at the pinnacle of Stockholm's fine 